# Volume 3: Discrete Hidden Markov Models
    <Name>
    <Class>
    <Date>

In [187]:
# Import needed libraries

!pip install hmmlearn 

import numpy as np
import copy
import string
import codecs
from hmmlearn import hmm

# Problems 1-5
This is the HMM class that you will be adding functions to throughout the lab.

In [188]:
class HMM(object):
    """
    Finite state space hidden Markov model.
    """
    
    # Problem 1
    def __init__(self, A, B, pi):
        """
        Initialize an HMM with parameters A, B, and pi.
        """
        # Initialize parameters
        self.A = A
        self.B = B
        self.pi = pi
    
    
    # Problem 2
    def forward_pass(self, z):
        """
        Compute the forward probability matrix.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        alpha : ndarray of shape (T, n)
            The forward probability matrix
        """
        
        # Get shapes and initialize forward probability matrix
        T = z.shape[0]
        n = self.A.shape[0]
        alpha_arrays = np.zeros((T,n))
        
        # Define function to calculate the alpha values
        def alpha_values(t, i, memo):
            if t ==0:
                return self.pi[i] * self.B[z[0],i]
            elif (t,i) in memo:
                return memo[(t,i)]
            else:
                # Do a recursive algorithm
                result = self.B[z[t], i] * np.sum([alpha_values(t-1, j, memo) * self.A[i,j] for j in range(n)])
                memo[(t,i)] = result # Cache the result
                return result
            
        memo = {}
        # Fill out the probability matrix      
        for t in range(T):
            for i in range(n):
                alpha_arrays[t,i] = alpha_values(t,i,memo)
        
        # Return the probability matrix   
        return alpha_arrays
        
    
    
    # Problem 4
    def backward_pass(self, z):
        """
        Compute the backward probability matrix and gamma values.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        beta : ndarray of shape (T, n)
            The backward probability matrix
        gamma : ndarray of shape (T, n)
            The state probability matrix
        """
        # Get the alpha values and the denominator values
        alpha_arrays = self.forward_pass(z)
        denom_prob = np.sum(alpha_arrays[-1, :])
        
        # Get shapes
        T = z.shape[0]
        n = self.A.shape[0]
        
        # Initialize beta arrays
        beta_arrays = np.zeros((T, n), dtype=float)  # Ensure the dtype is set to float

        # Define function to get beta arrays
        def beta_values(t, j, memo):
            if t == T - 1:
                return 1
            elif (t, j) in memo:
                return memo[(t, j)]
            else:
                # Do a recursive algorithm
                result = np.sum([self.A[i, j] * beta_values(t + 1, i, memo) * self.B[z[t + 1], i] for i in range(n)])
                memo[(t, j)] = result  # Cache the result
                return result

        memo = {}
        # Fill out the probability matrix
        for t in range(T):
            for i in range(n):
                beta_arrays[t,i] = beta_values(t, i, memo)
        
        # Calculate gamma arrays, return beta and gamma arrays
        gamma_arrays = (beta_arrays * alpha_arrays) / denom_prob
        return beta_arrays, gamma_arrays

        
    
    # Problem 5
    def viterbi_algorithm(self, z):
        """
        Compute the most likely hidden state sequence using the Viterbi algorithm.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        x*: ndarray of shape (T,)
            The most likely state sequence
        """
        # Get shapes; initialize nu_arrays
        T = z.shape[0]
        n = self.A.shape[0]
        nu_arrays = np.zeros((T,n))
        
        # Define nu_values function
        def nu_values(t,i,memo):
            if t == 0:
                return self.B[z[0],i] * self.pi[i]
            elif (t,i) in memo:
                return memo[(t,i)]
            else:
                # Do a recursive algorithm
                result = np.max([self.B[z[t],i] * self.A[i,j] * nu_values(t-1,j,memo) for j in range(n)])
                memo[(t,i)] = result # Cache the result
                return result
            
        memo = {}
        # Fill out the nu matrix
        for t in range(T):
            for i in range(n):
                nu_arrays[t,i] = nu_values(t,i,memo)
        
        # Initialize x_star array
        x_star_array = []
        
        # Calculate the x_star array
        for t in range(T-1,-1,-1):
            if t == T-1:
                x_star = np.argmax(nu_arrays[T-1,:])
                x_star_array.append(x_star)
            else:
                # Do a recursive algorithm
                x_star = np.argmax([self.A[x_star_array[-1], j] * nu_arrays[t, j] for j in range(n)])
                x_star_array.append(x_star)
        
        # Return x_star
        x_star_array = np.array(x_star_array)[::-1]
        return x_star_array

# Problem 2 test case

Use the following HMM and code to test your HMM class.
Compare the output to `forward_pass` with the lab pdf.

In [189]:
pi = np.array([.6, .4])
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
z_example = np.array([0, 1, 0, 2])
example_hmm = HMM(A, B, pi)

alpha=example_hmm.forward_pass(z_example)
print(np.sum(alpha[-1,:]))

0.009629599999999997


# Problem 3

Consider the following (very simplified) model of the price of a stock over time as an HMM.
The observation states will be the change in the value of the stock.
For simplicity, we will group these into five values: large decrease, small decrease, no change, small increase, large increase, labeled as integers from 0 to 4.
The hidden state will be the overall trends of the market.
We'll consider the market to have three possible states: declining in value (bear market), not changing in value (stagnant), and increasing in value (bull market), labeled as integers from 0 to 2.
Let the HMM modeling this scenario have parameters
$$
\boldsymbol\pi=\begin{bmatrix}
1/3 \\ 1/3 \\ 1/3
\end{bmatrix},
\quad
A=\begin{bmatrix}
0.5 & 0.3 & 0 \\
0.5 & 0.3 & 0.3 \\
0 & 0.4 & 0.7
\end{bmatrix},
\quad
B=\begin{bmatrix}
0.3 & 0.1 & 0 \\
0.3 & 0.2 & 0.1 \\
0.3 & 0.4 & 0.3 \\
0.1 & 0.2 & 0.4 \\
0 & 0.1 & 0.2
\end{bmatrix}
$$
The file `stocks.npy` contains a sequence of 50 observations drawn from this HMM.
What is the probability of this observation sequence given these model parameters?
Use your implementation of the forward pass algorithm from Problem 2 to find the answer.
Note that the answer is very small, because there are lots of possible observation sequences.

In [190]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

In [191]:
# Load in the stock data
stock_data = np.load('stocks.npy')

# Initialize the Problem 3 HMM
prob_3_hmm = HMM(A,B,pi)
alpha= prob_3_hmm.forward_pass(stock_data)
print(np.sum(alpha[-1,:]))


6.671115114537777e-34


# Problem 4

Create a method `backward_pass` in your HMM class to implement the backward pass algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return two arrays of the $\beta_t(i)$ and $\gamma_t(i)$ values.

Test your function on the example HMM, and compare the output with the lab pdf.

With your function and the stock model from Problem 3, answer the following question: given the observation sequence in `stocks.npy`, what is the most likely initial hidden state $X_0$?

In [192]:
# Test case; compare your output with what is in the lab pdf
beta, gamma = example_hmm.backward_pass(z_example)
print(beta)
print(gamma)

[[0.0302  0.02792]
 [0.0812  0.1244 ]
 [0.38    0.26   ]
 [1.      1.     ]]
[[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]


In [193]:
prob_3_hmm = HMM(A,B,pi)
beta, gamma = prob_3_hmm.backward_pass(stock_data)
print(beta, gamma)

[[2.60906852e-33 1.96928339e-33 1.43633541e-33]
 [9.28699665e-33 6.08009509e-33 3.36535236e-33]
 [3.43962839e-32 2.06377703e-32 4.23247583e-33]
 [1.82281050e-31 1.41082528e-31 9.93352593e-32]
 [7.28141015e-31 7.30598980e-31 7.92847436e-31]
 [1.75597024e-30 4.64745479e-30 7.34285927e-30]
 [2.44167336e-29 3.51194048e-29 4.49234080e-29]
 [1.84934008e-28 1.51700332e-28 1.27933529e-28]
 [6.03574990e-28 4.71988798e-28 3.39499396e-28]
 [2.38746949e-27 2.45454566e-27 2.74609509e-27]
 [5.27776160e-27 1.59580525e-26 2.55515992e-26]
 [4.34405713e-26 1.05555232e-25 1.59892444e-25]
 [1.76031826e-25 3.46389800e-25 4.96818058e-25]
 [1.00499233e-24 1.25782209e-24 1.50481690e-24]
 [8.13750694e-24 5.98116987e-24 4.09266682e-24]
 [3.01389146e-23 1.80833488e-23 9.15554748e-24]
 [9.91977920e-23 3.05184916e-22 4.89434597e-22]
 [7.37501843e-22 1.98395584e-21 3.07082801e-21]
 [2.30729603e-21 6.22137042e-21 9.63409209e-21]
 [7.16743074e-21 1.94892449e-20 3.02312050e-20]
 [2.16543233e-20 6.08471458e-20 9.492991

1

# Problem 5
Creating a method `viterbi_algorithm` in your HMM class to implement the Viterbi algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return the most likely state sequence $\mathbf{x}^*$.

Test your function on the example HMM and compare output with the lab pdf.

Apply your function to the stock market HMM from Problem 3.
With the observaition sequence from `stocks.npy`, what is the most likely sequence of hidden states?
Is the initial state of the most likely sequence the same as the most likely initial state you found in Problem 4?

In [194]:
# Test case
xstar = example_hmm.viterbi_algorithm(z_example)
print(xstar)

[1 1 1 0]


In [195]:
prob_5_hmm = HMM(A,B,pi)
x_star = prob_5_hmm.viterbi_algorithm(stock_data)
print(x_star)

[0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 0 0 0 1 2 2 2 2 2
 2 2 2 2 2 2 1 0 0 0 0 0 1]


The most likely initial state is 0 in this problem. It is different than the most likely initial state because it is solving a different problem

# Problem 6

Train a `hmmlearn.hmm.CategoricalHMM` on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values (26 lower case characters and 1 whitespace character).
Use `n_iter=200` and `tol=1e-4`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [197]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    """
    Reads in the file and prepares it for use in an HMM.
    Returns:
        symbols (dict): a dictionary that maps characters to their integer values
        obs_sequence (ndarray): an array of integers representing the read-in text
    """
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case
    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)
    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}
    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)
    return symbols, obs_sequence

In [208]:
# Load in data
symbols, obs = prep_data('declaration.txt')

# Set the number of states and observation values
N = 2
M = len(set(obs))

# Create and fit the CategoricalHMM
h = hmm.CategoricalHMM(n_components=N, n_iter=200, tol=1e-4)

# Fit the model
h.fit(obs.reshape(-1,1))

# Get the emission probabilities matrix B
B = h.emissionprob_.T

for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}".format(symbols[i], *B[i,:]))






 , 0.0500, 0.2990
a, 0.0000, 0.1319
b, 0.0226, 0.0000
c, 0.0437, 0.0000
d, 0.0599, 0.0000
e, 0.0000, 0.2375
f, 0.0428, 0.0000
g, 0.0309, 0.0000
h, 0.0829, 0.0001
i, 0.0000, 0.1241
j, 0.0038, 0.0000
k, 0.0030, 0.0004
l, 0.0542, 0.0000
m, 0.0342, 0.0000
n, 0.1148, 0.0000
o, 0.0036, 0.1376
p, 0.0328, 0.0000
q, 0.0014, 0.0000
r, 0.1010, 0.0000
s, 0.1136, 0.0000
t, 0.1521, 0.0000
u, 0.0000, 0.0578
v, 0.0176, 0.0000
w, 0.0230, 0.0000
x, 0.0021, 0.0000
y, 0.0093, 0.0116
z, 0.0010, 0.0000


# Problem 7

Repeat the same calculation with `WarAndPeace.txt` with 2 hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [212]:
# Load in data
symbols, obs = prep_data('WarAndPeace.txt')

# Set the number of states and observation values
N = 2
M = len(set(obs))
h = hmm.CategoricalHMM(n_components=N, n_iter=200, tol=1e-4)

# Fit the model 
h.fit(obs.reshape(-1,1))

# Get the emission probabilities matrix B
B = h.emissionprob_.T

for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}".format(symbols[i], *B[i,:]))

 , 0.2146, 0.0877
а, 0.0000, 0.1760
б, 0.0250, 0.0000
в, 0.0655, 0.0000
г, 0.0296, 0.0000
д, 0.0385, 0.0000
е, 0.0180, 0.1427
ж, 0.0140, 0.0000
з, 0.0252, 0.0000
и, 0.0017, 0.1315
й, 0.0149, 0.0000
к, 0.0497, 0.0010
л, 0.0719, 0.0000
м, 0.0381, 0.0000
н, 0.0973, 0.0000
о, 0.0000, 0.2407
п, 0.0346, 0.0062
р, 0.0597, 0.0000
с, 0.0513, 0.0280
т, 0.0780, 0.0000
у, 0.0000, 0.0590
ф, 0.0018, 0.0003
х, 0.0111, 0.0000
ц, 0.0049, 0.0000
ч, 0.0167, 0.0038
ш, 0.0109, 0.0000
щ, 0.0047, 0.0000
ъ, 0.0003, 0.0003
ы, 0.0000, 0.0376
ь, 0.0009, 0.0433
э, 0.0000, 0.0066
ю, 0.0079, 0.0024
я, 0.0128, 0.0328
ё, 0.0000, 0.0001


*Write your analysis here.*

The consonants are the ones with nonzero probabilities in the left column and the vowels are the ones with nonzero probabilities in the right column. Thus the Cyrillic characters that appear to be vowels:

a, e, и, o, y, ы, ь, я, ё